In [1]:
library("reticulate")
use_python("/net/mraid14/export/data/users/eladch/tools/CO7/python3/3.7.5/bin/python", required=TRUE)
library("anndata")
sc = import("scanpy")
mc2 = import("metacells")

In [2]:
library("metacell")
library("devtools")
library("Matrix")
library("data.table")
library("tidyverse")

Loading required package: usethis

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.3     v dplyr   1.0.9
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.0     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::between()   masks data.table::between()
x tidyr::expand()    masks Matrix::expand()
x dplyr::filter()    masks stats::filter()
x dplyr::first()     masks data.table::first()
x dplyr::lag()       masks stats::lag()
x dplyr::last()      masks data.table::last()
x tidyr::pack()      masks Matrix::pack()
x readr::read_csv()  masks anndata::read_csv()
x purrr::transpose() masks data.table::transpose()
x tidyr::unpack()    masks Matrix::unpack()



In [3]:
##### TODO change mat_id and scdb_dir to what you need #####
scdb_dir = "/net/mraid14/export/tgdata/users/hernan/wd_NEW/scrna_db/"

mat_id = "mat.SynEmb3.h5ad"
mc_id = "mc.SynEmb3.h5ad"
######

In [4]:
# Import MC2 output into a fake mc object and save it (I'm also replacing the egc with the fractions)
MC2_metacell = anndata::read_h5ad(paste(scdb_dir,mc_id, sep = ""))
MC2_mat = anndata::read_h5ad(paste(scdb_dir,mat_id, sep = ""))

cells_mc = MC2_mat$obs$metacell
cells_mc = cells_mc + 1
cell_names = MC2_mat$obs_names
names(cells_mc) = cell_names
gene_names = MC2_metacell$var_names

outlier_cells = names(cells_mc)[cells_mc == 0]
cells_mc_filt = cells_mc[cells_mc > 0]

umis2 = as.matrix(MC2_metacell$X)
fractions = umis2 / rowSums(umis2)
fractions = t(fractions)

scdb_init(scdb_dir,force_reinit = T)
MC1_mat = scdb_mat("SynEmb3")
MC1_mc_embexe <- scdb_mc("embexe_recolored")

scdb_add_mc("SynEmb3_MC1", tgMCCov(cells_mc_filt, c(outlier_cells, setdiff(MC1_mat@cells, c(outlier_cells, names(cells_mc_filt)))), MC1_mat))
MC1_metacell = scdb_mc("SynEmb3_MC1")
colnames(fractions) = colnames(MC1_metacell@e_gc)
MC1_metacell@e_gc = fractions
MC1_metacell@color_key <- MC1_mc_embexe@color_key
MC1_metacell@cell_names <- names(MC1_metacell@mc)
scdb_add_mc(id = "SynEmb3_MC1", mc = MC1_metacell)

initializing scdb to /net/mraid14/export/tgdata/users/hernan/wd_NEW/scrna_db/

add batch counts

compute footprints

compute absolute ps

compute coverage ps



In [8]:
bad_genes <- rownames(MC2_metacell$var)[MC2_metacell$var$forbidden_gene]
feature_genes <- rownames(MC2_metacell$var)[MC2_metacell$var$top_feature_gene]

write.table(bad_genes, file = "SynEmbs3_bad_genes.txt", sep ="\t")
write.table(feature_genes, file = "SynEmbs3_feature_genes.txt", sep ="\t")

MC1_mat = scm_ignore_cells(scmat = MC1_mat, ig_cells = names(MC1_metacell@mc), reverse = T)
MC1_mat = scm_ignore_genes(scmat = MC1_mat, ig_genes = bad_genes, reverse = F)
scdb_add_mat(id = "SynEmb3", mat = MC1_mat)

T_vm = 0.1
T_top3 = 3
T_tot = 50
K = 100

mcell_add_gene_stat(mat_id = "SynEmb3", gstat_id = "SynEmb3", force = T)
mcell_gset_filter_varmean(gstat_id = "SynEmb3",gset_id = "SynEmb3_new", T_vm = T_vm, force_new = T)
mcell_gset_filter_cov(gstat_id = "SynEmb3", gset_id = "SynEmb3_new", T_tot = T_tot, T_top3 = T_top3)

gset_new = scdb_gset("SynEmb3_new")

mcell_add_cgraph_from_mat_bknn(mat_id = "SynEmb3",
                               gset_id = "SynEmb3_new", 
                               graph_id = "SynEmb3_new", 
                               K = K, 
                               dsamp = T)

Calculating gene statistics... 

will downsamp

done downsamp

will gen mat_n

done gen mat_n

done computing basic gstat, will compute trends



..done


will downsample the matrix, N= 1942

will build balanced knn graph on 7335 cells and 913 genes, this can be a bit heavy for >20,000 cells



## Project on MC1 embexe atlas

In [9]:
setwd("/net/mraid14/export/tgdata/users/hernan/wd_NEW/SynEmbs3/")
fig_dir <- "/net/mraid14/export/tgdata/users/hernan/wd_NEW/SynEmbs3/figs/"
if(!dir.exists(fig_dir)) {
    dir.create(fig_dir)
  }

scfigs_init(fig_dir)

In [10]:
wt_atlas = mcell_gen_atlas(mat_id = "embexe",
                           mc_id = "embexe_recolored",
                           gset_id  = "embexe",
                           mc2d_id= "embexe_recolored_umap")

In [11]:
mcell_proj_on_atlas(mat_id = "SynEmb3", 
                    mc_id = "SynEmb3_MC1", 
                    atlas = wt_atlas, 
                    recolor_mc_id = "SynEmb3_MC1_recolored",
                    fig_cmp_dir = fig_dir,
                    max_entropy = 4, 
                    burn_cor = 0.6)

,1,2,3,4,5,6,7,8,9,10,⋯,354,355,356,357,358,359,360,361,362,363
1110017D15Rik,9.038285e-08,2.847618e-06,3.837344e-07,0.000000e+00,0.000000e+00,0.000000e+00,3.682900e-08,2.085789e-07,7.368029e-08,1.291184e-07,⋯,0.000000e+00,5.382700e-08,0.000000e+00,4.763224e-07,0.000000e+00,0.000000e+00,4.495539e-07,0.000000e+00,7.529465e-08,5.912135e-06
1700003E16Rik,9.349164e-07,2.075086e-06,5.010026e-07,5.291972e-08,0.000000e+00,0.000000e+00,7.493065e-07,8.105006e-07,4.927617e-07,9.313089e-07,⋯,0.000000e+00,7.964657e-07,0.000000e+00,5.527442e-07,0.000000e+00,0.000000e+00,9.305355e-07,0.000000e+00,0.000000e+00,3.818199e-06
1700007K13Rik,7.009188e-08,5.942178e-06,1.269752e-06,1.698576e-05,0.000000e+00,8.885533e-08,8.230875e-07,4.305611e-06,3.831250e-07,2.894430e-07,⋯,2.238824e-07,9.042994e-07,8.013297e-07,5.527442e-07,1.049064e-06,1.316545e-06,5.458159e-06,6.657076e-07,0.000000e+00,7.200575e-06
1700016K19Rik,1.254872e-07,3.238485e-06,1.311445e-06,1.721121e-06,0.000000e+00,0.000000e+00,1.231234e-06,5.337474e-07,3.817339e-07,3.685791e-07,⋯,0.000000e+00,1.003720e-06,3.124678e-07,1.828652e-07,5.333567e-08,2.594104e-07,4.878797e-07,9.451049e-08,7.529465e-08,5.727013e-06
1700088E04Rik,6.869978e-06,3.373163e-06,4.045513e-06,7.236805e-07,0.000000e+00,1.562984e-06,3.957545e-06,1.484876e-06,2.523993e-06,5.966762e-06,⋯,1.625798e-06,3.562009e-06,1.744549e-06,4.744648e-07,1.220223e-06,2.162996e-06,1.095481e-06,1.711846e-06,4.702858e-07,4.837309e-06
2200002D01Rik,1.168391e-05,4.882509e-06,1.057396e-05,9.224059e-05,5.838314e-06,1.323187e-05,1.111351e-05,4.445236e-06,1.350011e-05,1.329432e-05,⋯,1.349981e-05,1.083125e-05,9.498155e-06,8.876339e-06,1.680650e-05,7.862879e-06,5.775908e-06,2.445233e-05,1.184905e-05,5.098222e-06
2210409E12Rik,0.000000e+00,0.000000e+00,0.000000e+00,3.214533e-06,0.000000e+00,2.393795e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,⋯,1.117326e-07,0.000000e+00,2.387417e-07,0.000000e+00,5.462948e-07,0.000000e+00,0.000000e+00,2.288736e-07,4.143078e-07,0.000000e+00
2410004P03Rik,1.254872e-07,3.771153e-06,1.366995e-06,8.299533e-07,0.000000e+00,0.000000e+00,1.492770e-06,3.196015e-06,1.261936e-07,1.792674e-07,⋯,1.117326e-07,1.301209e-06,5.096793e-07,1.517266e-06,1.617286e-07,4.057396e-07,2.952083e-06,4.715093e-08,2.194976e-07,5.669743e-06
2610528J11Rik,6.683387e-06,5.219310e-06,0.000000e+00,2.152378e-05,7.213462e-07,1.122755e-07,0.000000e+00,8.283557e-06,2.656680e-06,6.341133e-06,⋯,1.117326e-07,0.000000e+00,2.327252e-07,1.828652e-07,2.705212e-07,2.670003e-07,6.836298e-06,4.937930e-07,6.844914e-08,3.946784e-06
2810417H13Rik,2.239880e-04,1.734786e-04,2.370644e-04,4.051406e-05,6.992382e-05,1.733511e-04,2.490720e-04,1.896952e-04,2.479293e-04,2.233822e-04,⋯,1.829662e-04,2.464679e-04,2.703710e-04,1.818801e-04,1.760221e-04,2.748841e-04,1.885871e-04,1.856792e-04,2.550770e-04,1.718321e-04


In [12]:
tgconfig::override_params(config_file = "../config/sing_emb.yaml",package = "metacell")

mcell_mc2d_force_knn(mc2d_id = "SynEmb3_MC1_recolored_umap",
                     mc_id = "SynEmb3_MC1_recolored",
                     graph_id = "SynEmb3_new")

mcell_mc2d_plot(mc2d_id = "SynEmb3_MC1_recolored_umap",
                plot_edges = T)

comp mc graph using the graph SynEmb3_new and K 35

Missing coordinates in some cells that are not ourliers or ignored - check this out! (total 51 cells are missing, maybe you used the wrong graph object? first nodes day6_plate1_MZG.day6_plate1_MZG162day6_plate1_MZG.day6_plate1_MZG288day6_plate1_MZG.day6_plate1_MZG308day6_plate3_MZG.day6_plate3_MZG062day6_plate3_MZG.day6_plate3_MZG198day6_plate3_MZG.day6_plate3_MZG208day6_plate3_MZG.day6_plate3_MZG252day6_plate3_MZG.day6_plate3_MZG369day6_plate3_MZG.day6_plate3_MZG374day6_plate4_MZG.day6_plate4_MZG167



png 
  2

In [14]:
mcell_mc_plot_marks(mc_id = "SynEmb3_MC1_recolored", gset_id="SynEmb3_new", mat_id="SynEmb3")

setting fig h to 1000 md levels 0 num of marks 913

